**Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd

**Loading Dataset**

In [ ]:
anime = pd.read_csv('anime.csv')
rating = pd.read_csv('rating.csv')

**EDA**

In [ ]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [ ]:
rating.tail()

,user_id,anime_id,rating
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9
7813736,73516,8074,9


**Data Cleaning**

In [ ]:
anime.isnull().sum()

,0
anime_id,0
name,0
genre,62
type,25
episodes,0
rating,230
members,0


In [ ]:
rating.isnull().sum()

,0
user_id,0
anime_id,0
rating,0


In [ ]:
anime['type'] = anime['type'].fillna('None')
anime['genre'] = anime['genre'].fillna('None')
anime['rating'] = anime['rating'].fillna(anime['rating'].median())
anime.isnull().sum()

,0
anime_id,0
name,0
genre,0
type,0
episodes,0
rating,0
members,0


In [ ]:
len(anime)

12294

In [ ]:
len(rating)

7813737

**Similarity Scores using Eucledian Distance**

In [ ]:
count = len(rating) #Taking the length of rating into a variable count
preferences = {} #Initialising a dictionary "preferences"

In [ ]:
#Numpy array
rating_array = rating.values
for i in range(count):
    user = rating_array[i, 0]
    anime = rating_array[i, 1]
    rating = rating_array[i, 2]

    if user not in preferences.keys():
        preferences[user] = {}

    if rating != -1: #Dealing with the -1 ratings(Anime watched but not rated)
        preferences[user][anime] = rating

In [ ]:
def sim_euclidian(preferences, person1,person2):

    similarity = {}

    #getting similar anime watched

    for anime in preferences[person1]:
        if anime in preferences[person2]:
            similarity[anime] = 1

    if len(similarity) == 0:
        return 0

    Euclidian_distance = np.sqrt(np.sum([np.power(preferences[person1][anime] - preferences[person2][anime],2)
                                 for anime in preferences[person1] if anime in preferences[person2]]))

    return 1/(1 + Euclidian_distance)

In [ ]:
def top_matches(preferences, person, n = 5, similarity = sim_euclidian):

    scores = [(similarity(preferences,person,reviewer),reviewer) for reviewer in preferences if reviewer != person]

    scores.sort()
    scores.reverse()
    return scores[:n]

In [ ]:
top_match = top_matches(preferences,1)
top_match

[(1.0, 73507), (1.0, 73481), (1.0, 73473), (1.0, 73440), (1.0, 73439)]

**Prediction Function**

In [ ]:
def recommendations(preferences, person, similarity = sim_euclidian):
    total = {}
    simsum = {}

    for reviwer in preferences:
        if reviwer == person:
            continue
        sim = similarity(preferences,person,reviwer)

        if sim <= 0:
            continue
        for anime in preferences[reviwer]:

            if anime not in preferences[person] or preferences[person][anime] == 0:
                total.setdefault(anime,0)
                total[anime] += preferences[reviwer][anime] * sim

                simsum.setdefault(anime,0)
                simsum[anime] += sim

    ranks = [(total/simsum[anime],anime) for anime,total in total.items()]

    ranks.sort()
    ranks.reverse()

    return ranks[:10],person

In [ ]:
anime = pd.read_csv('anime.csv')
anime_array = anime.values

In [ ]:
anime_array[0][0]

32281

In [ ]:
top_rec,user = recommendations(preferences,1)

In [ ]:
anime_list = []

for i in top_rec:
    for j in range(len(anime_array)):
        if i[1] == anime_array[j][0]:
            anime_list.append(anime_array[j])

In [ ]:
print("Top Recommendations for you:\n")
for i in anime_list:
    print("Anime :",i[1])
    print("Genre :",i[2])
    print("-"*50)

Top Recommendations for you:

Anime : STAR BEAT!: Hoshi no Kodou
Genre : Music
--------------------------------------------------
Anime : Shiroi Zou
Genre : Action, Historical, Kids
--------------------------------------------------
Anime : Dededen
Genre : Slice of Life
--------------------------------------------------
Anime : Trapp Ikka Monogatari Specials
Genre : Drama, Historical, Music, Romance
--------------------------------------------------
Anime : Konna Watashitachi ga Nariyuki de Heroine ni Natta Kekka www
Genre : Comedy, Fantasy
--------------------------------------------------
Anime : gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?
Genre : Comedy, Fantasy
--------------------------------------------------
Anime : Meitantei Holmes: Mrs. Hudson Hitojichi Jiken no Maki / Dover Kaikyou no Daikuuchuusen no Maki
Genre : Adventure, Comedy, Mystery
--------------------------------------------------
Anime : Only You: Viva! Cabaret Club
Genre : Comedy
------------------------------